# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter   #modules if used to save pd dataframe as clsx file
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN 

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [ ]:
for symbol in stocks['Ticker'][:1]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url)


## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [9]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        

symbol_groups = list(chunks(stocks['Ticker'], 100))   # get 100 chunks of data into a list
# print(symbol_groups)
symbol_strings = []
for i in range(0, len(symbol_groups)):                #  for each set of data, make one string for all 100 datas seperated by ',' and store it in symbol strings
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [45]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
         final_dataframe = final_dataframe.append(
         pd.Series([
             symbol,
             data[symbol]['quote']['latestPrice'],
             data[symbol]['quote']['peRatio'],
             'N/A'             
         ],
         index=my_columns),
         ignore_index = True)
            


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [46]:
# to get top 50 stocks and remove glamour stocks
# first sort the peRatio in asccending order (lower peRation = more earning)
# remove all pe ratios with negative value as it indicates the Earning is in negative
# slice dataframe to get top 50 data and reset index

final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]

final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [43]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [47]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [50]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row,['Number of Shares to Buy']] = math.floor(position_size/final_dataframe.loc[row,'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,33.670,2.04,5940
1,NLOK,20.720,4.04,9652
2,AIV,4.743,4.5,42167
3,UNM,23.010,4.91,8691
4,BIO,584.130,5.13,342
5,KIM,15.252,6.77,13113
6,MET,48.129,7.6,4155
7,ALL,110.505,7.7,1809
8,BIIB,250.840,8.25,797
9,CPB,47.340,8.3,4224


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [64]:
symbol = 'AAPL'
batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_url).json()
# print(data[symbol]['advanced-stats'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit



Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [83]:
rv_columns = [
    'Ticker', 
    'Price', 
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio', 
    'PE Percentile',
    'Price-to-book ratio',
    'PB Percentile',
    'Price-to-sales ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'   
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'    
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        
        # Price-to-earnings ratio
        pe_ratio = data[symbol]['quote']['peRatio']

        # Price-to-book ratio
        pb_ratio = data[symbol]['advanced-stats']['priceToBook']

        # Price-to-sales ratio
        ps_ratio = data[symbol]['advanced-stats']['priceToSales']

        
        # Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        try:   # to avoid getting NoneType error for missing values
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        
        # Enterprise Value divided by Gross Profit (EV/GP)
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                pe_ratio,
                'N/A',
                pb_ratio,
                'N/A',
                ps_ratio, 
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'RV Score'   
            ],
            index = rv_columns),
        ignore_index = True)
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.900,N/A,54.17,N/A,7.54,N/A,7.21,N/A,34.808228,N/A,13.438000,N/A,RV Score
1,AAL,17.580,N/A,-1.2,N/A,-1.86,N/A,0.4253,N/A,-6.152799,N/A,1.413274,N/A,RV Score
2,AAP,161.100,N/A,24.11,N/A,2.9,N/A,1.14,N/A,11.282611,N/A,2.544188,N/A,RV Score
3,AAPL,128.900,N/A,40.06,N/A,34.88,N/A,7.95,N/A,27.808843,N/A,20.863305,N/A,RV Score
4,ABBV,108.331,N/A,23.4,N/A,12.5,N/A,4.69,N/A,15.891210,N/A,9.372078,N/A,RV Score
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.140,N/A,33.49,N/A,-4,N/A,6,N/A,23.674366,N/A,10.022570,N/A,RV Score
501,ZBH,158.600,N/A,-210.22,N/A,2.7,N/A,4.47,N/A,25.739181,N/A,7.507945,N/A,RV Score
502,ZBRA,384.300,N/A,43.14,N/A,10.73,N/A,4.66,N/A,29.684103,N/A,11.003547,N/A,RV Score
503,ZION,43.610,N/A,18.2,N/A,0.9704,N/A,2.28,N/A,9.834966,N/A,2.608126,N/A,RV Score


None


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [86]:
## Dealing With Missing Data in Our DataFrame
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,219.540,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
71,BRK.B,229.259,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
118,CTL,11.000,N/A,10.03,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
136,DISCK,26.110,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
165,ETFC,49.420,N/A,14.7,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
168,EVRG,55.276,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
190,FOX,28.870,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
192,FRC,139.010,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
194,FTI,9.896,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score
204,GOOG,1761.430,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,RV Score


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [91]:
# filling empty values with  mean of other values in that column 
for column in ['Price-to-Earnings Ratio', 'Price-to-book ratio','Price-to-sales ratio','EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [92]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [100]:
from scipy.stats import percentileofscore as ps
metrics = {
    'Price-to-Earnings Ratio' : 'PE Percentile',
    'Price-to-book ratio' : 'PB Percentile',
    'Price-to-sales ratio' : 'PS Percentile',
    'EV/EBITDA' : 'EV/EBITDA Percentile',
    'EV/GP' : 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row,metrics[metric]] = ps(rv_dataframe[metric],rv_dataframe.loc[row,metric])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.900,N/A,54.17,85.7426,7.5400,72.5743,7.2100,81.0891,34.808228,85.7426,13.438000,81.1881,RV Score
1,AAL,17.580,N/A,-1.20,14.2574,-1.8600,4.55446,0.4253,3.16832,-6.152799,2.77228,1.413274,5.54455,RV Score
2,AAP,161.100,N/A,24.11,45.7426,2.9000,40.5941,1.1400,17.5248,11.282611,27.7228,2.544188,13.2673,RV Score
3,AAPL,128.900,N/A,40.06,73.6634,34.8800,96.4356,7.9500,83.9604,27.808843,80,20.863305,94.6535,RV Score
4,ABBV,108.331,N/A,23.40,44.5545,12.5000,85.1485,4.6900,67.8218,15.891210,48.5149,9.372078,63.7624,RV Score
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.140,N/A,33.49,66.1386,-4.0000,4.15842,6.0000,75.7426,23.674366,73.2673,10.022570,67.9208,RV Score
501,ZBH,158.600,N/A,-210.22,1.9802,2.7000,38.0198,4.4700,61.7822,25.739181,77.2277,7.507945,47.7228,RV Score
502,ZBRA,384.300,N/A,43.14,76.9307,10.7300,83.3663,4.6600,67.3267,29.684103,82.7723,11.003547,70.8911,RV Score
503,ZION,43.610,N/A,18.20,34.6535,0.9704,9.50495,2.2800,38.7129,9.834966,20.9901,2.608126,13.4653,RV Score


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [108]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics:
        value_percentiles.append(rv_dataframe.loc[row,metrics[metric]])
    rv_dataframe.loc[row,'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.900,N/A,54.17,85.7426,7.5400,72.5743,7.2100,81.0891,34.808228,85.7426,13.438000,81.1881,81.2673
1,AAL,17.580,N/A,-1.20,14.2574,-1.8600,4.55446,0.4253,3.16832,-6.152799,2.77228,1.413274,5.54455,6.05941
2,AAP,161.100,N/A,24.11,45.7426,2.9000,40.5941,1.1400,17.5248,11.282611,27.7228,2.544188,13.2673,28.9703
3,AAPL,128.900,N/A,40.06,73.6634,34.8800,96.4356,7.9500,83.9604,27.808843,80,20.863305,94.6535,85.7426
4,ABBV,108.331,N/A,23.40,44.5545,12.5000,85.1485,4.6900,67.8218,15.891210,48.5149,9.372078,63.7624,61.9604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.140,N/A,33.49,66.1386,-4.0000,4.15842,6.0000,75.7426,23.674366,73.2673,10.022570,67.9208,57.4455
501,ZBH,158.600,N/A,-210.22,1.9802,2.7000,38.0198,4.4700,61.7822,25.739181,77.2277,7.507945,47.7228,45.3465
502,ZBRA,384.300,N/A,43.14,76.9307,10.7300,83.3663,4.6600,67.3267,29.684103,82.7723,11.003547,70.8911,76.2574
503,ZION,43.610,N/A,18.20,34.6535,0.9704,9.50495,2.2800,38.7129,9.834966,20.9901,2.608126,13.4653,23.4653


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [114]:
rv_dataframe.sort_values('RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50] # get 50 cheapest stock 
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,22.276,N/A,4.93,15.6436,0.4259,4.9505,0.3974,2.77228,2.495128,3.16832,0.381597,0.39604,5.38614
1,AIG,38.750,N/A,-6.89,10.6931,0.5357,5.74257,0.7413,6.93069,4.406683,4.15842,0.720178,1.58416,5.82178
2,AAL,17.580,N/A,-1.20,14.2574,-1.8600,4.55446,0.4253,3.16832,-6.152799,2.77228,1.413274,5.54455,6.05941
3,MET,47.604,N/A,7.95,16.4356,0.5964,6.13861,0.6551,5.74257,4.194159,3.9604,0.616808,1.18812,6.69307
4,HFC,25.960,N/A,-26.63,6.73267,0.7924,7.72277,0.2913,1.38614,6.233472,6.53465,2.171946,11.4851,6.77228
5,F,9.524,N/A,-17.52,8.11881,1.1600,12.8713,0.2737,1.18812,4.728670,4.35644,2.133408,11.0891,7.52475
6,HIG,47.510,N/A,9.87,18.2178,0.9601,9.10891,0.8382,8.71287,3.905771,3.76238,0.797959,2.17822,8.39604
7,ALL,108.770,N/A,7.50,16.2376,1.3000,16.3366,0.7265,6.53465,2.504099,3.36634,0.715865,1.38614,8.77228
8,BA,227.250,N/A,-28.90,6.53465,-11.1700,2.9703,2.2000,36.7327,-31.132417,1.38614,-223.998662,0.19802,9.56436
9,PFG,49.490,N/A,11.37,20.396,0.8560,8.11881,0.8995,9.90099,7.145468,9.30693,0.873659,2.57426,10.0594


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [116]:
portfolio_input()

Enter the value of your portfolio:2500000


In [122]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row,'Price'])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,22.276,2244,4.93,15.6436,0.4259,4.9505,0.3974,2.77228,2.495128,3.16832,0.381597,0.39604,5.38614
1,AIG,38.750,1290,-6.89,10.6931,0.5357,5.74257,0.7413,6.93069,4.406683,4.15842,0.720178,1.58416,5.82178
2,AAL,17.580,2844,-1.20,14.2574,-1.8600,4.55446,0.4253,3.16832,-6.152799,2.77228,1.413274,5.54455,6.05941
3,MET,47.604,1050,7.95,16.4356,0.5964,6.13861,0.6551,5.74257,4.194159,3.9604,0.616808,1.18812,6.69307
4,HFC,25.960,1926,-26.63,6.73267,0.7924,7.72277,0.2913,1.38614,6.233472,6.53465,2.171946,11.4851,6.77228
5,F,9.524,5249,-17.52,8.11881,1.1600,12.8713,0.2737,1.18812,4.728670,4.35644,2.133408,11.0891,7.52475
6,HIG,47.510,1052,9.87,18.2178,0.9601,9.10891,0.8382,8.71287,3.905771,3.76238,0.797959,2.17822,8.39604
7,ALL,108.770,459,7.50,16.2376,1.3000,16.3366,0.7265,6.53465,2.504099,3.36634,0.715865,1.38614,8.77228
8,BA,227.250,220,-28.90,6.53465,-11.1700,2.9703,2.2000,36.7327,-31.132417,1.38614,-223.998662,0.19802,9.56436
9,PFG,49.490,1010,11.37,20.396,0.8560,8.11881,0.8995,9.90099,7.145468,9.30693,0.873659,2.57426,10.0594


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [126]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [127]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [128]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [129]:
writer.save()